In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
df = pd.read_excel("data-big.xlsx")

In [14]:
df.head()

,step_count,active_time,recommendation,run_step_count,update_time,create_time,speed,distance,calorie,walk_step_count,deviceuuid,healthy_step
0,1424,790945,6000,16,2020-05-06 06:52:53.223,2020-05-06 06:52:53.223,2.916667,1069.7500,59.170002,1408,FPoYR4lF9W,0
1,1584,937723,6000,21,2020-05-06 06:52:53.376,2020-05-06 06:52:53.376,2.500000,1190.7600,64.259995,1563,FPoYR4lF9W,0
2,2223,1237672,6000,1,2020-05-06 06:53:50.571,2020-05-06 06:53:50.571,1.777778,1678.0200,89.790000,2222,FPoYR4lF9W,0
3,5469,3081986,6000,19,2020-05-06 06:53:50.761,2020-05-06 06:53:50.761,2.583333,4148.0500,219.099990,5450,FPoYR4lF9W,2674
4,4242,2443635,6000,0,2020-05-06 06:53:50.888,2020-05-06 06:53:50.888,1.861111,3160.0398,171.439960,4242,FPoYR4lF9W,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2931 entries, 0 to 2930
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   step_count       2931 non-null   int64         
 1   active_time      2931 non-null   int64         
 2   recommendation   2931 non-null   int64         
 3   run_step_count   2931 non-null   int64         
 4   update_time      2931 non-null   datetime64[ns]
 5   create_time      2931 non-null   datetime64[ns]
 6   speed            2931 non-null   float64       
 7   distance         2931 non-null   float64       
 8   calorie          2931 non-null   float64       
 9   walk_step_count  2931 non-null   int64         
 10  deviceuuid       2931 non-null   object        
 11  healthy_step     2931 non-null   int64         
dtypes: datetime64[ns](2), float64(3), int64(6), object(1)
memory usage: 274.9+ KB


In [16]:
df.columns

Index(['step_count', 'active_time', 'recommendation', 'run_step_count',
       'update_time', 'create_time', 'speed', 'distance', 'calorie',
       'walk_step_count', 'deviceuuid', 'healthy_step'],
      dtype='object')

In [17]:
df.shape

(2931, 12)

In [18]:
df.isnull().sum()
df['active_time'] = df['active_time'] / 60000

In [19]:
np.random.seed(42)

selected_columns = ['calorie' , 'step_count',
                     'active_time']

In [20]:
df_selected = df[selected_columns]

In [21]:
df_selected.head()

,calorie,step_count,active_time
0,59.170002,1424,13.182417
1,64.259995,1584,15.628717
2,89.790000,2223,20.627867
3,219.099990,5469,51.366433
4,171.439960,4242,40.727250


In [22]:
df_selected = df_selected.dropna()

In [23]:
df_selected.head()

,calorie,step_count,active_time
0,59.170002,1424,13.182417
1,64.259995,1584,15.628717
2,89.790000,2223,20.627867
3,219.099990,5469,51.366433
4,171.439960,4242,40.727250


In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_selected)

In [26]:
df_scaled

array([[-0.80283253, -0.8085275 , -0.80565096],
       [-0.75243497, -0.74264417, -0.69659329],
       [-0.49965468, -0.47952263, -0.47372789],
       ...,
       [-0.2212301 , -0.25181337, -0.28852868],
       [-1.04115689, -1.05270759, -1.05507753],
       [-0.39915651, -0.39140368, -0.40723583]])

In [27]:
from sklearn.ensemble import IsolationForest

In [28]:
model = IsolationForest(contamination=0.05)  # Adjust contamination based on expected anomaly rate
model.fit(df_scaled)

IsolationForest(contamination=0.05)

In [29]:
import joblib

In [7]:
import numpy as np
import joblib

model = joblib.load('isolation_forest_model.joblib')
scaler = joblib.load('scaler.joblib')

def check_for_anomaly(data: list):
    selected_columns = ['calorie' , 'step_count', 'active_time']
    data = np.array(data).reshape((1, -1))
    data_scaled = scaler.transform(data)
    prediction = model.predict(data_scaled)

    if prediction == 1:
        return  {'message': 'Anomaly detected!'}
    else:
        return {'message': 'No anomaly detected!'}


In [15]:
check_for_anomaly([300, 2000, 60])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


{'message': 'Anomaly detected!'}